In [ ]:
!pip install transformers datasets pdfminer.six PyMuPDF nltk spacy streamlit wordcloud
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 60.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving uri_en.pdf to uri_en.pdf


In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Use the uploaded filename
pdf_path = "uri_en.pdf"
raw_text = extract_text_from_pdf(pdf_path)

print(raw_text[:1000])  # show first 1000 chars


Uniform Resource Identifier (URI)
Péter Jeszenszky
Faculty of Informatics, University of Debrecen
jeszenszky.peter@inf.unideb.hu
Last modified: September 20, 2025
 
 
2
URI (1)
●Uniform Resource Identifier (URI):
– A compact sequence of characters that identifies 
an abstract or physical resource.
●A resource is not necessarily available on the Web.
●URIs can be assigned even to objects from the real 
world or to concepts.
●Current standard:
– Tim Berners-Lee, Roy Fielding, Larry Masinter. 
Uniform Resource Identifier (URI): Generic 
Syntax. RFC 3986, January 2005. 
https://www.rfc-editor.org/rfc/rfc3986
 
 
3
URI (2)
●Each URI begins with a scheme name that is 
separated by a ':' character from the 
scheme-specific part of the URI.
– Scheme specifications can define their scheme-
specific syntax within certain limits.
●The organization responsible for the 
administration of the URI schemes:
– Internet Assigned Numbers Authority (IANA) 
https://www.iana.org/
●See: Uniform Resource Iden

In [ ]:
import re

def clean_text(raw_text):
    # remove citations like [1], (Smith 2020)
    text = re.sub(r'\[.*?\]|\(.*?\)', '', raw_text)
    # remove LaTeX style equations ($...$ or \[...\])
    text = re.sub(r'\$.*?\$', '', text)
    text = re.sub(r'\\\[.*?\\\]', '', text)
    # collapse multiple spaces/newlines
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

cleaned_text = clean_text(raw_text)
print(cleaned_text[:1000])


Uniform Resource Identifier Péter Jeszenszky Faculty of Informatics, University of Debrecen jeszenszky.peter@inf.unideb.hu Last modified: September 20, 2025 2 URI ●Uniform Resource Identifier : – A compact sequence of characters that identifies an abstract or physical resource. ●A resource is not necessarily available on the Web. ●URIs can be assigned even to objects from the real world or to concepts. ●Current standard: – Tim Berners-Lee, Roy Fielding, Larry Masinter. Uniform Resource Identifier : Generic Syntax. RFC 3986, January 2005. https://www.rfc-editor.org/rfc/rfc3986 3 URI ●Each URI begins with a scheme name that is separated by a ':' character from the scheme-specific part of the URI. – Scheme specifications can define their scheme- specific syntax within certain limits. ●The organization responsible for the administration of the URI schemes: – Internet Assigned Numbers Authority https://www.iana.org/ ●See: Uniform Resource Identifier Schemes https://www.iana.org/assignments/

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')   # <-- NEW



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
sentences = nltk.sent_tokenize(cleaned_text)
print("First 5 sentences:", sentences[:5])


First 5 sentences: ['Uniform Resource Identifier Péter Jeszenszky Faculty of Informatics, University of Debrecen jeszenszky.peter@inf.unideb.hu Last modified: September 20, 2025 2 URI ●Uniform Resource Identifier : – A compact sequence of characters that identifies an abstract or physical resource.', '●A resource is not necessarily available on the Web.', '●URIs can be assigned even to objects from the real world or to concepts.', '●Current standard: – Tim Berners-Lee, Roy Fielding, Larry Masinter.', 'Uniform Resource Identifier : Generic Syntax.']


In [ ]:
import pandas as pd

df = pd.DataFrame({"sentences": sentences})
df.to_csv("processed_uri_en.csv", index=False)
print("Saved processed paper with", len(df), "sentences.")


Saved processed paper with 135 sentences.


**# Week 2: Baseline Summarization**

In [ ]:
!pip install sumy


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 69.4 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21695 sha256=80d4fd30c9726ada939f0d706e1ff030abdcf7714b4daab753c29cb48b880a22
  Stored in directory: /root/.cache/pip/wheels/32/99/64/59305409cacd03aa03e7bddf31a9db34b1fa7033bd41972662
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=f1452c42c4211cb6a46f5800c10e6c99893f8d77e28031f42cffdf388fcc2c9a
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built breadability docopt


Step 2: Implement TextRank **Summarizer**

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer   # <-- notice text_rank not textrank


In [ ]:
# 📌 Imports
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer  # fixed import
from rouge_score import rouge_scorer
import textstat
import pandas as pd

# 📌 Load your preprocessed CSV
df = pd.read_csv("processed_uri_en.csv")

# Join sentences into one text
full_text = " ".join(df["sentences"].astype(str))

# 📌 TextRank Summarizer
def textrank_summary(text, num_sentences=5):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    summary = summarizer(parser.document, num_sentences)
    return [str(sentence) for sentence in summary]

# Generate summary
summary_sentences = textrank_summary(full_text, num_sentences=5)

print("🔹 Baseline Summary (TextRank):")
for i, sent in enumerate(summary_sentences, 1):
    print(f"{i}. {sent}")

# 📌 Save summary to CSV
summary_df = pd.DataFrame({"summary": summary_sentences})
summary_df.to_csv("summary_textrank.csv", index=False)
print("✅ Saved baseline summary to summary_textrank.csv")

# 📌 Evaluate (ROUGE + Readability)
reference = df["sentences"].iloc[0]  # placeholder reference
candidate = " ".join(summary_sentences)

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference, candidate)

print("\n📊 Evaluation:")
print("ROUGE Scores:", scores)
print("Readability Score (Flesch-Kincaid):", textstat.flesch_reading_ease(candidate))


🔹 Baseline Summary (TextRank):
1. RFC 3305, August 2002. https://www.rfc-editor.org/rfc/rfc3305 – URIs, URLs, and URNs: Clarifications and Recommendations 1.0—Report from the joint W3C/IETF URI Planning Interest Group (W3C Note, 21 September 2001) https://www.w3.org/TR/uri-clarification/ 8 URL vs URN ●According to the contemporary view, a URI can be a locator, a name, or both at the same time.
2. https://html.spec.whatwg.org/multipage/semantics.html#the-base-element ●If no base URI is embedded and a representation is enclosed within another entity – for example, another document –, then the base URI is the base URI of the entity in which the representation is encapsulated.
3. ●If no base URI is embedded and the representation is not encapsulated within some other entity, then, if a URI was used to retrieve the representation, that URI is considered the base URI.
4. 32 Relative Reference Resolution Examples ●Let http://example/a/b/c?q be the base URI Relative Reference Target URI d http

**🔹 Step 3: Evaluate the Summary🔹 Step 3: Evaluate the Summary**

In [ ]:
!pip install rouge-score textstat


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.2/239.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.0 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=4f68c95db8470417dc76d94ab4cfb4f27b710fa5666e79eb7dc79612d61d9b13
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer
import textstat

# Compare summary vs first abstract sentence (proxy for evaluation)
reference = df["sentences"].iloc[0]  # just a placeholder for now
candidate = " ".join(summary_sentences)

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference, candidate)

print("ROUGE Scores:", scores)
print("Readability Score (Flesch-Kincaid):", textstat.flesch_reading_ease(candidate))


ROUGE Scores: {'rouge1': Score(precision=0.05067567567567568, recall=0.38461538461538464, fmeasure=0.08955223880597016), 'rougeL': Score(precision=0.02702702702702703, recall=0.20512820512820512, fmeasure=0.04776119402985075)}
Readability Score (Flesch-Kincaid): 20.081278280543017


**🔹 Step 4: Save the Summary**

In [ ]:
summary_df = pd.DataFrame({"summary": summary_sentences})
summary_df.to_csv("summary_textrank.csv", index=False)
print("Saved baseline summary.")


Saved baseline summary.


**Week-3**

**🔹 Step 1: Install Hugging Face Transformers**

In [ ]:
!pip install transformers sentencepiece


*🔹 Step 2: Load Pretrained T5 Mode*

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

**🔹 Step 3: Summarize with T5**

In [ ]:
def t5_summarize(text, max_input_length=512, max_output_length=150):
    # Truncate long text (T5 has 512 token limit)
    inputs = tokenizer("summarize: " + text,
                       return_tensors="pt",
                       max_length=max_input_length,
                       truncation=True)

    summary_ids = model.generate(inputs["input_ids"],
                                 num_beams=4,
                                 min_length=30,
                                 max_length=max_output_length,
                                 early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

t5_summary = t5_summarize(full_text)
print("🔹 T5 Summary:\n", t5_summary)


🔹 T5 Summary:
 Uniform Resource Identifier Péter Jeszenszky Faculty of Informatics, University of Debrecen jeszenszky.peter@inf.unideb.hu Last modified: September 20, 2025 2 URI. URIs can be assigned even to objects from the real world or concepts.


In [ ]:
with open("summary_t5.txt", "w") as f:
    f.write(t5_summary)
print("✅ Saved T5 summary to summary_t5.txt")


✅ Saved T5 summary to summary_t5.txt


**🔹 Step 1: Install Transformers (if not already)**

In [ ]:
!pip install transformers sentencepiece


**🔹 Step 2: Load PEGASUS Model**

In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

model_name = "google/pegasus-arxiv"   # scientific summarization model
pegasus_tokenizer = PegasusTokenizer.from_pretrained(model_name)
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:02<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-arxiv and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

****🔹 Step 3: Summarize with PEGASUS

In [ ]:
def pegasus_summarize(text, max_input_length=512, max_output_length=150):
    inputs = pegasus_tokenizer(text,
                               truncation=True,
                               return_tensors="pt",
                               max_length=max_input_length)
    summary_ids = pegasus_model.generate(inputs["input_ids"],
                                         num_beams=5,
                                         min_length=30,
                                         max_length=max_output_length,
                                         early_stopping=True)
    return pegasus_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

pegasus_summary = pegasus_summarize(full_text)
print("🔹 PEGASUS Summary:\n", pegasus_summary)


🔹 PEGASUS Summary:
 in this paper , we present a method for the construction of a _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _


**🔹 Step 4: Save PEGASUS Summary**

In [ ]:
with open("summary_pegasus.txt", "w") as f:
    f.write(pegasus_summary)
print("✅ Saved PEGASUS summary to summary_pegasus.txt")


✅ Saved PEGASUS summary to summary_pegasus.txt


**🔹 Step 5: Compare All Three Models**

In [ ]:
print("📌 TextRank Summary:\n", " ".join(summary_sentences))
print("\n📌 T5 Summary:\n", t5_summary)
print("\n📌 PEGASUS Summary:\n", pegasus_summary)


📌 TextRank Summary:
 RFC 3305, August 2002. https://www.rfc-editor.org/rfc/rfc3305 – URIs, URLs, and URNs: Clarifications and Recommendations 1.0—Report from the joint W3C/IETF URI Planning Interest Group (W3C Note, 21 September 2001) https://www.w3.org/TR/uri-clarification/ 8 URL vs URN ●According to the contemporary view, a URI can be a locator, a name, or both at the same time. https://html.spec.whatwg.org/multipage/semantics.html#the-base-element ●If no base URI is embedded and a representation is enclosed within another entity – for example, another document –, then the base URI is the base URI of the entity in which the representation is encapsulated. ●If no base URI is embedded and the representation is not encapsulated within some other entity, then, if a URI was used to retrieve the representation, that URI is considered the base URI. 32 Relative Reference Resolution Examples ●Let http://example/a/b/c?q be the base URI Relative Reference Target URI d http://example/a/b/d ./d h

**Week 4: Application Layer (Streamlit app).**

**🔹 Step 1: Install Streamlit & ngrok (for Colab use)**

In [ ]:
!pip install streamlit pyngrok


**Step 2: Write the Streamlit App (app.py)**

In [ ]:
%%writefile app.py
import streamlit as st
import fitz
import re
import nltk
import pandas as pd
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from transformers import T5Tokenizer, T5ForConditionalGeneration, PegasusTokenizer, PegasusForConditionalGeneration

# Load models
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")
pegasus_tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-arxiv")
pegasus_model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-arxiv")

nltk.download('punkt')
nltk.download('punkt_tab')

# --- Utility functions ---
def extract_text_from_pdf(uploaded_file):
    doc = fitz.open(stream=uploaded_file.read(), filetype="pdf")
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def clean_text(raw_text):
    text = re.sub(r'\[.*?\]|\(.*?\)', '', raw_text)   # remove citations
    text = re.sub(r'\$.*?\$', '', text)               # remove equations
    text = re.sub(r'\\\[.*?\\\]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def textrank_summary(text, num_sentences=5):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    summary = summarizer(parser.document, num_sentences)
    return " ".join(str(sentence) for sentence in summary)

def t5_summarize(text, max_input_length=512, max_output_length=150):
    inputs = t5_tokenizer("summarize: " + text,
                          return_tensors="pt",
                          max_length=max_input_length,
                          truncation=True)
    summary_ids = t5_model.generate(inputs["input_ids"],
                                    num_beams=4,
                                    min_length=30,
                                    max_length=max_output_length,
                                    early_stopping=True)
    return t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def pegasus_summarize(text, max_input_length=512, max_output_length=150):
    inputs = pegasus_tokenizer(text,
                               truncation=True,
                               return_tensors="pt",
                               max_length=max_input_length)
    summary_ids = pegasus_model.generate(inputs["input_ids"],
                                         num_beams=5,
                                         min_length=30,
                                         max_length=max_output_length,
                                         early_stopping=True)
    return pegasus_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# --- Streamlit UI ---
st.title("📄 Scientific Paper Simplifier")
st.write("Upload a scientific paper PDF and choose a model to generate a simplified summary.")

uploaded_file = st.file_uploader("Upload a PDF", type="pdf")

if uploaded_file:
    raw_text = extract_text_from_pdf(uploaded_file)
    cleaned = clean_text(raw_text)

    model_choice = st.selectbox("Choose a summarization model", ["TextRank", "T5", "PEGASUS"])

    if st.button("Summarize"):
        if model_choice == "TextRank":
            summary = textrank_summary(cleaned)
        elif model_choice == "T5":
            summary = t5_summarize(cleaned)
        else:
            summary = pegasus_summarize(cleaned)

        st.subheader("🔹 Simplified Summary")
        st.write(summary)


Overwriting app.py


**🔹 Step 3: Run the App in Colab**

In [ ]:
from pyngrok import ngrok

# Kill any previous tunnels
ngrok.kill()

# Start a new tunnel for port 8501 (Streamlit default)
public_url = ngrok.connect(8501)
print("🌍 Public URL:", public_url)


ERROR:pyngrok.process.ngrok:t=2025-09-30T13:16:02+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-09-30T13:16:02+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-09-30T13:16:02+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [ ]:
!streamlit run app.py &





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.74.208.148:8501

  Stopping...


In [ ]:
!ngrok config add-authtoken YOUR_TOKEN_HERE


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("🌍 Public URL:", public_url)
!streamlit run app.py &


ERROR:pyngrok.process.ngrok:t=2025-09-30T13:17:02+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: YOUR_TOKEN_HERE\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: YOUR_TOKEN_HERE\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.

In [ ]:
!ngrok config add-authtoken ngrok config add-authtoken 337UFDJ0Mdql7oMUHZmt5UERCTg_48H1sxmE8qvf2BrA1oUWr


ERROR:  accepts 1 arg(s), received 4


In [ ]:
!ngrok config add-authtoken 337UFDJ0Mdql7oMUHZmt5UERCTg_48H1sxmE8qvf2BrA1oUWr


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("🌍 Public URL:", public_url)
!streamlit run app.py &


🌍 Public URL: NgrokTunnel: "https://uncodified-dorene-nontraceably.ngrok-free.dev" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.74.208.148:8501



2025-09-30 13:17:43.229668: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759238263.285825    5262 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759238263.303082    5262 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759238263.350962    5262 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759238263.351022    5262 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759238263.351028    5262 computation_placer.cc:177] computation placer alr

In [ ]:
!streamlit run app.py &>/dev/null&


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("🌍 Public URL:", public_url)


In [ ]:
!streamlit run app.py &


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8502)
print("🌍 Public URL:", public_url)


In [ ]:
from rouge_score import rouge_scorer
import textstat
import pandas as pd

# --- Summaries from your models ---
textrank_result = textrank_summary(full_text, num_sentences=5)
t5_result = t5_summarize(full_text)
pegasus_result = pegasus_summarize(full_text)

# --- Convert TextRank list to string ---
textrank_text = " ".join(textrank_result)

# --- Reference text (use first abstract-like paragraph as proxy) ---
reference = df["sentences"].iloc[0] if not df.empty else full_text[:500]

# --- Initialize scorer ---
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

def evaluate_summary(reference, candidate):
    rouge = scorer.score(reference, candidate)
    readability = textstat.flesch_reading_ease(candidate)
    return rouge, readability

# Evaluate all three
results = {
    "TextRank": evaluate_summary(reference, textrank_text),
    "T5": evaluate_summary(reference, t5_result),
    "PEGASUS": evaluate_summary(reference, pegasus_result)
}

# Create DataFrame for comparison
eval_df = pd.DataFrame({
    "Model": [],
    "ROUGE-1": [],
    "ROUGE-L": [],
    "Readability": []
})

for model, (rouge, readability) in results.items():
    eval_df = pd.concat([eval_df, pd.DataFrame({
        "Model": [model],
        "ROUGE-1": [rouge['rouge1'].fmeasure],
        "ROUGE-L": [rouge['rougeL'].fmeasure],
        "Readability": [readability]
    })])

print("📊 Evaluation Results:")
print(eval_df)


In [ ]:
human_eval = {
    "TextRank": {"Clarity": 3, "Coverage": 2, "Usefulness": 3},
    "T5": {"Clarity": 4, "Coverage": 4, "Usefulness": 4},
    "PEGASUS": {"Clarity": 5, "Coverage": 5, "Usefulness": 5},
}


In [ ]:
import matplotlib.pyplot as plt

# Bar chart of ROUGE-L
plt.bar(eval_df["Model"], eval_df["ROUGE-L"])
plt.title("ROUGE-L Comparison")
plt.ylabel("Score")
plt.show()

# Bar chart of Readability
plt.bar(eval_df["Model"], eval_df["Readability"])
plt.title("Readability (Flesch Score)")
plt.ylabel("Score")
plt.show()
